# Pediatric Pneumonia Detection - Training Pipeline

This notebook demonstrates the end-to-end training process for detecting pneumonia in pediatric chest X-rays using a ResNet-50 based architecture.

## 1. Setup and Imports
We start by setting up the environment and importing the necessary modules from our `model_core` package.

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path to import model_core
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from model_core.data_pipeline import DataPipeline
from model_core.model_builder import ModelBuilder
from model_core.trainer import Trainer
from model_core.utils import Utils

# Configuration
DATASET_PATH = "/path/to/chest_xray"  # UPDATE THIS PATH
OUTPUT_DIR = "../outputs"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

## 2. Data Preparation
We use the `DataPipeline` class to explore the dataset and create data generators with augmentation.

In [ ]:
# Initialize pipeline
pipeline = DataPipeline(DATASET_PATH, img_size=IMG_SIZE, batch_size=BATCH_SIZE)

# Explore dataset structure
stats = pipeline.explore_dataset()

# Create stratified validation split
pipeline.create_validation_split(val_ratio=0.15)

# Create data generators
train_gen, val_gen, test_gen = pipeline.create_generators(use_augmentation=True)

# Calculate class weights for imbalance handling
class_weights = Utils.calculate_class_weights(train_gen)

### Visualize Samples
Let's look at some representative X-ray images from the dataset.

In [ ]:
pipeline.visualize_samples()

## 3. Training Stage 1: Feature Extraction
In this stage, we freeze the ResNet-50 backbone and only train the custom classification head.

In [ ]:
# Build model with frozen backbone
model = ModelBuilder.build(img_size=IMG_SIZE, trainable_backbone=False)
ModelBuilder.compile(model, learning_rate=1e-4)

# Initialize trainer
trainer = Trainer(model, output_dir=OUTPUT_DIR)

# Train Stage 1
history1 = trainer.train(
    train_gen, val_gen, 
    epochs=8,
    stage='stage1',
    class_weight=class_weights
)

# Visualize training history
trainer.plot_history(history1, 'stage1')
Utils.print_best_metrics(history1, 'Stage 1')

## 4. Training Stage 2: Fine-Tuning
Now we unfreeze the top layers of the backbone to fine-tune the feature representations.

In [ ]:
# Unfreeze layers
base_model = model.layers[1]
base_model.trainable = True
# Keep bottom layers frozen to prevent overfitting
for layer in base_model.layers[:140]:
    layer.trainable = False

# Recompile with lower learning rate
ModelBuilder.compile(model, learning_rate=1e-5)

# Train Stage 2
history2 = trainer.train(
    train_gen, val_gen, 
    epochs=5,
    stage='stage2',
    class_weight=class_weights
)

# Visualize fine-tuning history
trainer.plot_history(history2, 'stage2')
Utils.print_best_metrics(history2, 'Stage 2')